# Long-Term Semantic Memory with LangGraph

This notebook demonstrates how to implement long-term semantic memory using embeddings and vector search. 

## Key Concepts:
- **Semantic memory**: Stores important facts extracted from conversations
- **Embeddings**: Convert text to dense vectors for semantic similarity
- **Vector store**: Retrieves relevant facts based on semantic meaning
- **Memory extraction**: Automatically identify and store key information
- **Retrieval**: Find relevant stored facts for context in future conversations

In [1]:
# Install required dependencies
import subprocess
import sys

packages = [
    "langgraph",
    "langchain",
    "langchain-openai",
    "langchain-chroma",
    "python-dotenv"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

print("✓ All dependencies installed")

✓ All dependencies installed


In [2]:
import os
from dotenv import load_dotenv
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing_extensions import TypedDict

# Verify API key
api_key = " "
if not api_key:
    print("⚠️  Warning: OPENAI_API_KEY not found. Please set it in your .env file")
else:
    print("✓ OpenAI API key loaded")

✓ OpenAI API key loaded


## Step 1: Initialize Embeddings and Vector Store

Setup the semantic memory system with embeddings and a vector database.

In [5]:
try:
    # Initialize embeddings
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=api_key)
    
    # Initialize vector store (using Chroma for simplicity, stores in memory)
    vector_store = Chroma(
        collection_name="semantic_memory",
        embedding_function=embeddings,
    )
    
    print("✓ Embeddings and vector store initialized")
    
except Exception as e:
    print(f"✗ Error initializing embeddings: {e}")
    print("Make sure OPENAI_API_KEY is set")

✓ Embeddings and vector store initialized


## Step 2: Define the Agent State

The state now includes:
- `messages`: Current conversation
- `semantic_memory`: Stored facts from conversations (as text)

In [6]:
def add_messages(left: list[BaseMessage], right: list[BaseMessage]) -> list[BaseMessage]:
    """Append messages to existing messages."""
    return left + right

def add_memories(left: list[str], right: list[str]) -> list[str]:
    """Append memories to existing memories."""
    return left + right

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    semantic_memory: Annotated[list[str], add_memories]

print("✓ Agent state defined with short-term and long-term memory")

✓ Agent state defined with short-term and long-term memory


## Step 3: Create Agent Nodes

1. **extract_memory**: Extract important facts from user messages
2. **retrieve_memory**: Find relevant stored facts
3. **agent**: Process conversation with retrieved context

In [10]:
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, openai_api_key=api_key)
    
    def extract_memory(state: AgentState) -> dict:
        """
        Extract important facts from the latest user message.
        Store them in the vector store for future retrieval.
        """
        messages = state["messages"]
        
        # Get the last user message
        last_user_msg = None
        for msg in reversed(messages):
            if isinstance(msg, HumanMessage):
                last_user_msg = msg.content
                break
        
        if not last_user_msg:
            return {}
        
        print("🧠 Extracting facts from message...")
        
        # Use LLM to extract key facts with explicit criteria
        extraction_prompt = f"""Extract facts matching these criteria:
- Named entities (people, companies, places)
- Preferences (likes, dislikes)
- Skills or expertise mentioned
- Goals or projects
- Important dates or numbers

Message: {last_user_msg}

Format each fact on a new line as a simple statement. Only extract if there are facts matching the criteria."""
        
        facts_response = llm.invoke(extraction_prompt)
        facts_text = facts_response.content
        
        # Store each fact in vector store
        if facts_text.strip() and "no facts" not in facts_text.lower():
            facts = [f.strip() for f in facts_text.split("\n") if f.strip()]
            if facts:
                print("Storing {len(facts)} fact(s):")
                for fact in facts:
                    print(f"   • {fact}")
                vector_store.add_texts(facts)
                return {"semantic_memory": facts}
        
        print("⊘ No facts extracted")
        return {}
    
    def retrieve_memory(state: AgentState) -> dict:
        """
        Retrieve relevant facts from semantic memory based on current context.
        """
        messages = state["messages"]
        
        # Get the last message
        last_msg = messages[-1].content if messages else ""
        
        if not last_msg or len(vector_store.get()["ids"]) == 0:
            return {}
        
        print("🔍 Retrieving relevant facts from memory...")
        
        # Search for relevant facts
        results = vector_store.similarity_search(last_msg, k=3)
        
        if results:
            retrieved_facts = "\n".join([doc.page_content for doc in results])
            print(f"✓ Found {len(results)} relevant fact(s)")
            return {"semantic_memory": [f"[Retrieved] {retrieved_facts}"]}
        
        print("⊘ No relevant facts found")
        return {}
    
    def agent(state: AgentState) -> dict:
        """
        Agent that processes conversation with semantic memory context.
        """
        messages = state["messages"]
        semantic_memory = state.get("semantic_memory", [])
        
        # Build context from semantic memory
        context = ""
        if semantic_memory:
            context = f"\nRelevant facts from memory:\n" + "\n".join(semantic_memory)
        
        # Prepare system message with memory context
        system_msg = f"""You are a helpful assistant with access to long-term semantic memory.
        
{context}

Use the facts from memory to provide better context-aware responses."""
        
        # Call LLM with full context
        response = llm.invoke([
            {"role": "system", "content": system_msg},
            *messages
        ])
        
        return {"messages": [response]}
    
    print("✓ Agent nodes created successfully")

except Exception as e:
    print(f"✗ Error creating agent nodes: {e}")
    import traceback
    traceback.print_exc()

✓ Agent nodes created successfully


## Step 4: Build the Graph

The workflow:
1. Extract memory from user message
2. Retrieve relevant facts from semantic memory
3. Agent responds with memory-augmented context

In [11]:
# Create the graph
graph_builder = StateGraph(AgentState)

# Add nodes
graph_builder.add_node("extract_memory", extract_memory)
graph_builder.add_node("retrieve_memory", retrieve_memory)
graph_builder.add_node("agent", agent)

# Add edges
graph_builder.add_edge(START, "extract_memory")
graph_builder.add_edge("extract_memory", "retrieve_memory")
graph_builder.add_edge("retrieve_memory", "agent")
graph_builder.add_edge("agent", END)

# Compile the graph
graph = graph_builder.compile()

print("✓ Graph compiled successfully")

✓ Graph compiled successfully


## Step 5: Run a Multi-Turn Conversation Demo

Demonstrate how semantic memory persists across different conversations.

In [13]:
try:
    # Initialize conversation with semantic memory
    initial_state = {
        "messages": [],
        "semantic_memory": []
    }
    
    # Multi-turn conversation
    conversation_turns = [
        "I work as a software engineer at TechCorp and I love building AI systems.",
        "What do you know about my job?",
        "I'm starting a new project on recommendation systems.",
        "What are my interests and background?"
    ]
    
    print("=" * 60)
    print("LONG-TERM SEMANTIC MEMORY DEMONSTRATION")
    print("=" * 60)
    print()
    
    current_state = initial_state.copy()
    
    for i, user_input in enumerate(conversation_turns, 1):
        print(f"Turn {i}: User Input")
        print(f">> {user_input}")
        print()
        
        # Add user message to state
        current_state["messages"].append(HumanMessage(content=user_input))
        
        # Run the agent
        result = graph.invoke(current_state)
        current_state = result
        
        # Extract and display agent response
        last_message = current_state["messages"][-1]
        print(f"Turn {i}: Agent Response")
        print(f"<< {last_message.content}")
        print()
        
        # Show semantic memory size
        stored_count = len(vector_store.get()["ids"])
        print(f"📚 Semantic memory facts stored: {stored_count}")
        print("-" * 60)
        print()
    
    print("✓ Multi-turn conversation completed")
    print(f"✓ Total facts stored in semantic memory: {len(vector_store.get()['ids'])}")

except Exception as e:
    print(f"✗ Error running conversation: {e}")
    import traceback
    traceback.print_exc()

LONG-TERM SEMANTIC MEMORY DEMONSTRATION

Turn 1: User Input
>> I work as a software engineer at TechCorp and I love building AI systems.

🧠 Extracting facts from message...
💾 Storing 3 fact(s):
   • - Named entity: TechCorp
   • - Preference: loves building AI systems
   • - Skill or expertise: software engineer
🔍 Retrieving relevant facts from memory...
✓ Found 3 relevant fact(s)
Turn 1: Agent Response
<< That's great to hear! Working as a software engineer at TechCorp must provide you with exciting opportunities, especially since you love building AI systems. Are there any specific projects or technologies you're currently working on in the AI field?

📚 Semantic memory facts stored: 19
------------------------------------------------------------

Turn 2: User Input
>> What do you know about my job?

🧠 Extracting facts from message...
💾 Storing 1 fact(s):
   • No specific facts matching the criteria have been provided in the message.
🔍 Retrieving relevant facts from memory...
✓ Found 

## Key Takeaways: Long-Term Semantic Memory Pattern

### How it Works:
1. **Extract**: Important facts are automatically extracted from user messages using LLM
2. **Embed**: Facts are converted to embeddings for semantic understanding
3. **Store**: Embeddings are stored in a vector database (Chroma)
4. **Retrieve**: When responding, relevant facts are retrieved using semantic similarity
5. **Contextualize**: Retrieved facts augment the agent's context for better responses

### Characteristics:
- ✅ Remembers important facts across multiple conversations
- ✅ Retrieves semantically similar facts (not just keyword matching)
- ✅ Facts persist if vector store is persistent
- ✅ Scales better than storing all messages
- ✅ More cost-effective for long-term conversations
- ❌ Requires extraction logic (LLM calls)
- ❌ Information loss during fact extraction

### Common Use Cases:
- Personal assistant that remembers user preferences
- Customer support with historical context
- Research assistant that builds knowledge base
- Agent that evolves understanding over time